# Задания 6-7

Исследуйте зависимость числа итераций $T(n, k)$,
необходимых градиентному спуску для сходимости в зависимости от размерности пространства $2 \leqslant n \leqslant 10^{3}$
и числа обусловленности оптимизируемой функции $1 \leqslant k \leqslant 10^{3}$.